# bestiaire spectral : comment s'y retrouver ?




## création du dashboard spectro
- lancer la cellule suivante
- sur 'Colormap', bouton droit : "create new view for cell output"
- redimensionner ou déplacer l'onglet créé 'Output View' 

In [66]:
%matplotlib widget
import numpy as np
from spectro_dashboard import SpectroDashboard

# 1. Afficher le dashboard
db = SpectroDashboard()
db.show()


# étoiles


<img src="hr_diag.jpeg" alt="diagramme HR" width="1000" height="800">


Pour un même type spectral, la classe de luminosité (de I à V) détermine la luminosité :
- Naines (V) : Petites, gravité de surface importante $\rightarrow$ Pression élevée $\rightarrow$ ; Les atomes se cognent souvent $\rightarrow$ Raies larges (élargissement collisionnel).
- Supergéantes (I) : Immenses, atmosphère très diluée $\rightarrow$ Pression faible $\rightarrow$ Raies fines et profondes.

### le diagramme HR

### Type O: Chaudes et Bleues
- Température : $> 30\,000 \text{ K}$
- Le continuum bleu est très intense.
- L'hydrogène est faible car il est presque totalement ionisé.
- Le Marqueur : Présence d'Hélium ionisé (He II 4200, 4542, 4686), signature exclusive des étoiles O.
- exemples :
    - Naine (V)	10 Lacertae (O9V)	Raies d'absorption de l'Hélium visibles mais "floues" à cause de la pression.



In [187]:
from astropy import units as u
from specutils import Spectrum

db.clear_spectra()
filename = 'data/MilesLibraryBASS/09Ib s0252 HD057061.dat'
_spc_array = np.loadtxt(filename)
_spec1d = Spectrum(spectral_axis=_spc_array[:,0] * u.Unit('Angstrom'), flux=_spc_array[:,1] * u.Unit('adu'))
db.load_spectrum(_spec1d.wavelength, _spec1d.flux, label='09Ib')



09Ib : dispersion=0.9000 Å/px


In [186]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from pathlib import Path    
 

# 1. Chargement des données
data = np.loadtxt(filename) # Remplacez par votre fichier M !
wvl = data[:, 0]
flux = data[:, 1]

# 2. Normalisation
flux_norm = flux / np.max(flux)

# --- C'EST ICI QUE TOUT SE JOUE : Extraction du Continuum ---
# On découpe le spectre en tranches et on prend le max de chaque tranche.
# Cela simule votre œil qui relie les sommets.
nb_bins = 50
x_continuum = []
y_continuum = []

# On ne prend que les données où il y a du signal (> 3500A souvent)
mask_signal = wvl > 3500
x_vals = wvl[mask_signal]
y_vals = flux_norm[mask_signal]

x_bins = np.array_split(x_vals, nb_bins)
y_bins = np.array_split(y_vals, nb_bins)

for x_chunk, y_chunk in zip(x_bins, y_bins):
    if len(y_chunk) > 0:
        max_idx = np.argmax(y_chunk)
        # On garde le point le plus haut de la tranche
        x_continuum.append(x_chunk[max_idx])
        y_continuum.append(y_chunk[max_idx])

x_continuum = np.array(x_continuum)
y_continuum = np.array(y_continuum)

# 3. Fonction de Planck manuelle (pour éviter les erreurs d'unités)
def planck_math(wav_angstrom, T, scale):
    h = 6.626e-34
    c = 2.9979e8
    k = 1.3806e-23
    lam = wav_angstrom * 1e-10
    term1 = (2 * h * c**2) / (lam**5)
    try:
        exponent = (h * c) / (lam * k * T)
        term2 = 1.0 / (np.exp(np.clip(exponent, 0, 700)) - 1.0)
    except:
        return 0.0
    return scale * term1 * term2

# 4. Le Fit (Sur les points de continuum UNIQUEMENT)
valeur_ref = planck_math(6000, 33500, 1.0)
scale_init = 0.5 / valeur_ref if valeur_ref > 0 else 1e-10
p0 = [33500, scale_init] # On part de 3500 K

try:
    # On contraint T entre 2000K et 50000K
    popt, pcov = curve_fit(planck_math, x_continuum, y_continuum, p0=p0, bounds=([2000, 0], [50000, np.inf]))
    temp_mesuree = popt[0]
    scale_final = popt[1]
except RuntimeError:
    print("Fit échoué.")
    temp_mesuree = 0
    scale_final = 0

# 5. Graphique
#plt.figure(figsize=(10, 6))

# Spectre complet
#db.load_spectrum(wvl, flux_norm, label='09Ib')

#plt.plot(wvl, flux_norm, color='gray', alpha=0.5, label='Spectre complet')

# Points de Continuum utilisés (les sommets)
#plt.scatter(x_continuum, y_continuum, color='orange', s=20, zorder=5, label='Points Continuum')

# Courbe de Planck ajustée
y_fit = planck_math(wvl, temp_mesuree, scale_final)
#plt.plot(wvl, y_fit, color='red', linewidth=2, label=f'Fit Planck (T={temp_mesuree:.0f} K)')
#db.load_spectrum(x_continuum, y_continuum, label=f"Points Continuum")

db.load_spectrum(wvl, y_fit, label=f"Fit Planck (T={temp_mesuree:.0f} K)")

#plt.xlabel("Longueur d'onde (Angstrom)")
#plt.ylabel("Flux Normalisé")
#plt.title(f"{Path(filename).name} : {temp_mesuree:.0f} K")
#plt.legend()
#plt.ylim(0, 1.1)
#plt.show()

09Ib : dispersion=0.9000 Å/px
Fit Planck (T=2006 K) : dispersion=0.9000 Å/px



### Type B : L'Hélium Neutre
- Température : $10\,000 - 30\,000 \text{ K}$
- Les raies de l'Hydrogène (Balmer) deviennent plus visibles.
- Le Marqueur : Maximum de l'Hélium neutre ($\text{He I}$) (4026, 4471, 5876, 6678) : Signature des étoiles B.
- exemples:
    - Naine (V)Regulus (B7V) Raies d'Hélium larges.
    - Supergéante (I)Rigel (B8I) Raies d'Hélium beaucoup plus fines et nettes. C'est l'exemple classique pour montrer l'effet de pression.

In [3]:
from astropy import units as u
from specutils import Spectrum

db.clear_spectra()
_spc_array = np.loadtxt('data/MilesLibraryBASS/B9.5V s0823 HD209459.dat')
_spec1d = Spectrum(spectral_axis=_spc_array[:,0] * u.Unit('Angstrom'), flux=_spc_array[:,1] * u.Unit('adu'))
db.load_spectrum(_spec1d.wavelength, _spec1d.flux, label='B9.5V')

_spc_array = np.loadtxt('data/MilesLibraryBASS/B9 s0430 HD105262.dat')
_spec1d = Spectrum(spectral_axis=_spc_array[:,0] * u.Unit('Angstrom'), flux=_spc_array[:,1] * u.Unit('adu'))
db.load_spectrum(_spec1d.wavelength, _spec1d.flux, label='B9')


B9.5V : dispersion=0.9000 Å/px
B9 : dispersion=0.9000 Å/px


### Type A : Le Royaume de l'Hydrogène
- Température : $7\,500 - 10\,000 \text{ K}$
- Le Marqueur : Les raies de l'Hydrogène (Série de Balmer : $H\alpha, H\beta, H\gamma$) sont à leur intensité maximale.
- Exemples:
    - Naine (V)	Véga ou Sirius (A0V/A1V)	Les raies de l'hydrogène sont très larges avec des "ailes" étendues (broad wings).
    - Supergéante (I)	Deneb (A2I)	Les raies de l'hydrogène sont étroites et profondes, sans les ailes larges typiques des naines.

In [4]:
from astropy import units as u
from specutils import Spectrum

db.clear_spectra()

_spc_array = np.loadtxt('data/MilesLibraryBASS/A0 s0085 HD014829.dat')
_spec1d = Spectrum(spectral_axis=_spc_array[:,0] * u.Unit('Angstrom'), flux=_spc_array[:,1] * u.Unit('adu'))
db.load_spectrum(_spec1d.wavelength, _spec1d.flux, label='AO')

_spc_array = np.loadtxt('data/MilesLibraryBASS/A0 V (HB) s0446 HD109995.dat')
_spec1d = Spectrum(spectral_axis=_spc_array[:,0] * u.Unit('Angstrom'), flux=_spc_array[:,1] * u.Unit('adu'))
db.load_spectrum(_spec1d.wavelength, _spec1d.flux, label='A0V')


AO : dispersion=0.9000 Å/px
A0V : dispersion=0.9000 Å/px


### Type F : La Transition Métallique
- Température : $6\,000 - 7\,500 \text{ K}$
- Le Marqueur : L'hydrogène faiblit.
- Le Calcium ionisé ($\text{Ca II}$, raies H et K) devient fort.
- Les métaux neutres ($\text{Fe I}, \text{Cr}$) apparaissent.
- exemples :
    - Naine (V)	Procyon (F5V)	Spectre "propre" avec des métaux modérés.
    - Supergéante (I)	Canopus (F0I)	Les raies de métaux ionisés (Ti II, Fe II) sont renforcées par rapport aux naines.


In [5]:
from astropy import units as u
from specutils import Spectrum

db.clear_spectra()

_spc_array = np.loadtxt('data/MilesLibraryBASS/F0 s0345 HD081029.dat')
_spec1d = Spectrum(spectral_axis=_spc_array[:,0] * u.Unit('Angstrom'), flux=_spc_array[:,1] * u.Unit('adu'))
db.load_spectrum(_spec1d.wavelength, _spec1d.flux, label='F0')

_spc_array = np.loadtxt('data/MilesLibraryBASS/F0 V s0910 HD029375.dat')
_spec1d = Spectrum(spectral_axis=_spc_array[:,0] * u.Unit('Angstrom'), flux=_spc_array[:,1] * u.Unit('adu'))
db.load_spectrum(_spec1d.wavelength, _spec1d.flux, label='F0V')


F0 : dispersion=0.9000 Å/px
F0V : dispersion=0.9000 Å/px


### Type G : Les Étoiles de Type Solaire
- Température : $5\,000 - 6\,000 \text{ K}$
- Le Marqueur : Les raies H et K du $\text{Ca II}$ sont énormes.
- La Bande G (molécule CH) est très visible.
- exemples :
    - Naine (V)	Soleil (G2V)	La bande moléculaire CN (Cyanogène) est faible.
    - Géante (III)	Capella (G5III)	La bande du Cyanogène (CN) est forte dans le violet. C'est un critère discriminant majeur pour les géantes G et K.

In [6]:
from astropy import units as u
from specutils import Spectrum

db.clear_spectra()

_spc_array = np.loadtxt('data/MilesLibraryBASS/G0V s0176 HD034411.dat')
_spec1d = Spectrum(spectral_axis=_spc_array[:,0] * u.Unit('Angstrom'), flux=_spc_array[:,1] * u.Unit('mJy'))
db.load_spectrum(_spec1d.wavelength, _spec1d.flux, label='G0V')

_spc_array = np.loadtxt('data/MilesLibraryBASS/G2Iab s0313 HD074395.dat')
_spec1d = Spectrum(spectral_axis=_spc_array[:,0] * u.Unit('Angstrom'), flux=_spc_array[:,1] * u.Unit('mJy'))
db.load_spectrum(_spec1d.wavelength, _spec1d.flux, label='G2Iab')



G0V : dispersion=0.9000 Å/px
G2Iab : dispersion=0.9000 Å/px


### Type K : L'Arrivée des Molécules
- Température : $3\,500 - 5\,000 \text{ K}$
- Le Marqueur : Forêt de raies métalliques. Apparition des bandes moléculaires d'oxyde de titane ($\text{TiO}$), TiO (4765, 6158, 7050) et Mg I (5175)
- exemples :
    - Naine (V)	61 Cygni (K5V)	Raies métalliques très encombrées.
    - Géante (III)	Arcturus (K1III)	Comme pour les G, le CN est plus fort. Le profil global semble plus "accidenté" dans le bleu.

In [7]:
from astropy import units as u
from specutils import Spectrum

db.clear_spectra()

_spc_array = np.loadtxt('data/MilesLibraryBASS/K0 s0187 HD037828.dat')
_spec1d = Spectrum(spectral_axis=_spc_array[:,0] * u.Unit('Angstrom'), flux=_spc_array[:,1] * u.Unit('adu'))
db.load_spectrum(_spec1d.wavelength, _spec1d.flux, label='K0')

_spc_array = np.loadtxt('data/MilesLibraryBASS/K0 V s0724 HD185144.dat')
_spec1d = Spectrum(spectral_axis=_spc_array[:,0] * u.Unit('Angstrom'), flux=_spc_array[:,1] * u.Unit('adu'))
db.load_spectrum(_spec1d.wavelength, _spec1d.flux, label='K0V')


K0 : dispersion=0.9000 Å/px
K0V : dispersion=0.9000 Å/px


### Type M : Les Étoiles Froides
- Température : $< 3\,500 \text{ K}$
- Le Marqueur : Le spectre est "mangé" par d'énormes bandes d'absorption moléculaires ($\text{TiO}$), donnant un aspect cannelé (en dents de scie).
- exemples :
    - Naine (V) Proxima Centauri (M5V) Présence de bandes de CaH (Hydrure de Calcium) et parfois $\text{MgH}$. C'est la signature de la haute gravité des naines rouges.
    - Supergéante (I) Bételgeuse (M1-M2I) Bandes de TiO énormes, mais absence quasi-totale de CaH.

In [8]:
from astropy import units as u
from specutils import Spectrum

db.clear_spectra()

_spc_array = np.loadtxt('data/MilesLibraryBASS/M0V s0339 HD079211.dat')
_spec1d = Spectrum(spectral_axis=_spc_array[:,0] * u.Unit('Angstrom'), flux=_spc_array[:,1] * u.Unit('adu'))
db.load_spectrum(_spec1d.wavelength, _spec1d.flux, label='M0V')

_spc_array = np.loadtxt('data/MilesLibraryBASS/M0.5 IIb s0535 HD132933.dat')
_spec1d = Spectrum(spectral_axis=_spc_array[:,0] * u.Unit('Angstrom'), flux=_spc_array[:,1] * u.Unit('adu'))
db.load_spectrum(_spec1d.wavelength, _spec1d.flux, label='M05II')


M0V : dispersion=0.9000 Å/px
M05II : dispersion=0.9000 Å/px


### Méthode d'identification rapide
1. Regarder l'Hydrogène : Fort $\rightarrow$ A.
2. Regarder l'Hélium : Présent $\rightarrow$ O ou B.
3. Regarder les bandes moléculaires : Présentes $\rightarrow$ K ou M.
4. Pour la luminosité (I vs V) : Regarder la largeur des raies (surtout pour B et A) ou la présence d'éléments sensibles à la gravité comme le Sr II ou le CN (pour F, G, K).


ATTENTION au "Piège" des Naines Rouges (Type M):
- géantes rouges (comme Bételgeuse) et petites naines rouges (comme l'étoile de Barnard) ont toutes deux des bandes de TiO.
- regarder dans le rouge profond (~6380-6800 $\mathring{A}$) : Si bandes de CaH (Hydrure de Calcium) -> c'est une petite étoile compacte (Naine) car le CaH survit mal dans la faible gravité des géantes.


Sr II (4078) : La raie du Strontium :
- Forte = Supergéante (Basse pression).
- Faible = Naine (Haute pression).

CaH (6385) : Hydrure de Calcium :
- Présent = Naine Rouge (M V).
- Absent = Géante Rouge (M III).



# nebuleuses

In [9]:
from astropy import units as u
from specutils import Spectrum

db.clear_spectra()

_spec1d = Spectrum.read('data/plouis/_m57_20240710_865.fit')
db.load_spectrum(_spec1d.wavelength, _spec1d.flux, label='M57')

#_spec1d = Spectrum.read('data/plouis/ngc6543_20161006_849.fit')
#db.load_spectrum(_spec1d.wavelength, _spec1d.flux, label='ngc6543')


M57 : dispersion=0.8775 Å/px


# galaxies

In [67]:
db.clear_spectra()

_spc_array = np.loadtxt('data/plouis/_ngc4151_20170121_974_P.Louis.dat')
_spec1d = Spectrum(spectral_axis=_spc_array[:,0] * u.Unit('Angstrom'), flux=_spc_array[:,1] * u.Unit('adu'))
db.load_spectrum(_spec1d.wavelength, _spec1d.flux, label='ngc4151')

_spc_array = np.loadtxt('data/plouis/_ngc7215-_20161228_896_P.Louis.dat')
_spec1d = Spectrum(spectral_axis=_spc_array[:,0] * u.Unit('Angstrom'), flux=_spc_array[:,1] * u.Unit('adu'))
db.load_spectrum(_spec1d.wavelength, _spec1d.flux, label='ngc7215')

_spc_array = np.loadtxt('data/plouis/_3c273_20170407_870_PLouis.dat')
_spec1d = Spectrum(spectral_axis=_spc_array[:,0] * u.Unit('Angstrom'), flux=_spc_array[:,1] * u.Unit('adu'))
db.load_spectrum(_spec1d.wavelength, _spec1d.flux, label='3c273')

# on affiche des barres verticales pour marquer les raies de H
#h_a = db.ax_spec.axvline(x=6563, color='red', linestyle='--', alpha=0.6, linewidth=4)
#h_b = db.ax_spec.axvline(x=4861, color='red', linestyle='--', alpha=0.6, linewidth=4)

import astropy.units as u
from astropy.constants import c

lambda_obs = 5631 * u.AA # Longueur d'onde observée
lambda_rest = 4861 * u.AA # Longueur d'onde de H-alpha au repos

# 1. Calcul du Redshift (z)
z = (lambda_obs - lambda_rest) / lambda_rest
print(f"{z=:.4f}")

vitesse_radiale_classique = (z * c).to(u.km/u.s)
print(f"{vitesse_radiale_classique=:.0f}")

vitesse_radiale_relativiste = lambda_obs.to(u.km/u.s, equivalencies=u.doppler_relativistic(lambda_rest))
print(f"{vitesse_radiale_relativiste=:.0f}")

#h_a.remove()
#h_b.remove()


ngc4151 : dispersion=0.8083 Å/px
ngc7215 : dispersion=0.8083 Å/px
3c273 : dispersion=0.8421 Å/px
z=0.1584
vitesse_radiale_classique=47488 km / s
vitesse_radiale_relativiste=43767 km / s


La relation entre le décalage spectral (redshift, $z$) et la longueur d'onde est :$$z = \frac{\lambda_{obs} - \lambda_{rest}}{\lambda_{rest}}$$Et la relation entre le redshift et la vitesse radiale ($V_r$) dépend de la convention choisie :Optique (non-relativiste, pour $V_r \ll c$):$$V_r \approx c \cdot z = c \cdot \frac{\lambda_{obs} - \lambda_{rest}}{\lambda_{rest}}$$Relativiste :$$V_r = c \cdot \frac{(\lambda_{obs}/\lambda_{rest})^2 - 1}{(\lambda_{obs}/\lambda_{rest})^2 + 1}$$